In [1]:
!ls

gdrive	sample_data


In [1]:
!pip install pytorch_transformers

     |████████████████████████████████| 143kB 3.4MB/s 
     |████████████████████████████████| 1.0MB 25.7MB/s 


In [0]:
import os
os.chdir("/content/gdrive/My Drive/AVP")


In [9]:
import torch
from pytorch_transformers import *
import pandas as pd
# PyTorch-Transformers has a unified API
# for 6 transformer architectures and 27 pretrained weights.
#          Model          | Tokenizer          | Pretrained weights shortcut
MODELS = [(BertModel,       BertTokenizer,      'bert-base-uncased'),
          (OpenAIGPTModel,  OpenAIGPTTokenizer, 'openai-gpt'),
          (GPT2Model,       GPT2Tokenizer,      'gpt2'),
          (TransfoXLModel,  TransfoXLTokenizer, 'transfo-xl-wt103'),
          (XLNetModel,      XLNetTokenizer,     'xlnet-base-cased'),
          (XLMModel,        XLMTokenizer,       'xlm-mlm-enfr-1024')]
MODELS[0]

(pytorch_transformers.modeling_bert.BertModel,
 pytorch_transformers.tokenization_bert.BertTokenizer,
 'bert-base-uncased')

In [3]:

# Let's encode some text in a sequence of hidden-states using each model:
for model_class, tokenizer_class, pretrained_weights in MODELS:
    # Load pretrained model/tokenizer
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)

    # Encode text
    input_ids = torch.tensor([tokenizer.encode("Here is some text to encode")])
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples



100%|██████████| 830121242/830121242 [00:15<00:00, 52782597.90B/s]


In [0]:
def give_feature(model:tuple,df:pd.DataFrame):
    tokenizer = model[1].from_pretrained('bert-large-cased')
    model = model[0].from_pretrained('bert-large-cased')
    
    def return_feat(x):
      input_ids = torch.tensor([tokenizer.encode(x)])
      with torch.no_grad():
          last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples
      return last_hidden_states
    feat=[]
    for _,row in df.iterrows():
      feat.append(return_feat(row['text']))
    return torch.tensor(feat)

In [0]:
df=pd.read_csv('train_F3WbcTw.csv')

In [22]:
resFeat=give_feature(MODELS[0],df)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 1338740706/1338740706 [00:29<00:00, 45217265.48B/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (2904 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: ignored

In [0]:
# Each architecture is provided with several class for fine-tuning on down-stream tasks, e.g.
BERT_MODEL_CLASSES = [BertModel, BertForPreTraining, BertForMaskedLM, BertForNextSentencePrediction,
                      BertForSequenceClassification, BertForMultipleChoice, BertForTokenClassification,
                      BertForQuestionAnswering]

# All the classes for an architecture can be initiated from pretrained weights for this architecture
# Note that additional weights added for fine-tuning are only initialized
# and need to be trained on the down-stream task
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
for model_class in BERT_MODEL_CLASSES:
    # Load pretrained model/tokenizer
    model = model_class.from_pretrained('bert-base-uncased')

# Models can return full list of hidden-states & attentions weights at each layer
model = model_class.from_pretrained(pretrained_weights,
                                    output_hidden_states=True,
                                    output_attentions=True)
input_ids = torch.tensor([tokenizer.encode("Let's see all hidden-states and attentions on this text")])
all_hidden_states, all_attentions = model(input_ids)[-2:]

# Models are compatible with Torchscript
model = model_class.from_pretrained(pretrained_weights, torchscript=True)
traced_model = torch.jit.trace(model, (input_ids,))

# Simple serialization for models and tokenizers
model.save_pretrained('./directory/to/save/')  # save
model = model_class.from_pretrained('./directory/to/save/')  # re-load
tokenizer.save_pretrained('./directory/to/save/')  # save
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)



In [2]:
train.text.apply(lambda x:len(x))

NameError: ignored